In [1]:
import os
import torch
import numpy as np
import torchvision
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as T
import torch.nn.functional as F

from ig_pkg.datasets import get_datasets

from ig_pkg.models.generator import get_model
from ig_pkg.models.classifier import get_classifier
from ig_pkg.models.pretrained_models import get_pretrained_model

from ig_pkg.inputattribs.ig import make_interpolation, ig
from ig_pkg.inputattribs.baseline_generator import get_baseline_generator

from ig_pkg.misc import process_heatmap, normalize_tensor, convert_to_img, convert_mask_img, label_to_class, tran, na_imshow

import time
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from ig_pkg.loss.focal_loss import FocalLoss
from ig_pkg.loss.metrics import ArcMarginProduct, AddMarginProduct

import torchvision.models as models
from torch.autograd import Variable


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

my_cmap=plt.cm.seismic(np.arange(plt.cm.seismic.N))
device = 'cuda:2'

In [2]:
import torchvision.transforms as T
CIFAR10_STATS = {
    'mean' : [0.4914, 0.4822, 0.4465],
    'std' : [0.2023, 0.1994, 0.2010]
}

transform = T.Compose([
                T.ToTensor(), 
                T.Normalize(CIFAR10_STATS['mean'], CIFAR10_STATS['std'])
            ])

valid_dataset = torchvision.datasets.CIFAR10(root='/data8/donghun/cifar10/untracked/', train=False, transform=transform)
# valid_dataset = torchvision.datasets.CIFAR10(root='/root/data/cifar10', train=False, transform=transform)
# interpolation = torch.from_numpy(np.load('/root/results/cifar10/image_simple_gradient_ascent_interpolation.npy'))

# interpolation.shape
model = torch.jit.load("/data8/donghun/cifar10/results/densenet/script_model.pt",  map_location='cpu').eval().to(device)
# classifier = torch.jit.load("/root/data/cifar10/cifar10/results/densenet/script_model.pt",  map_location='cpu').eval().to('cuda:0')

# ae = torch.jit.load("/root/data/cifar10/cifar10/results/ae/script_model.pt",  map_location='cpu').eval().to('cuda:0')
# bottle = torch.load("/root/ig_inversion/results/bottleneck_ae/results/model_best.pt",  map_location='cpu').eval().to('cuda:0')

bottleneck = torch.load("/home/dhlee/code/ig_inversion/ig_pkg/bottleneck_ae/results/cifar10_8/model_best.pt",  map_location='cpu').eval().to(device)

In [17]:
a = np.linspace(-1, 1, 5)
b = np.linspace(-1, 1, 5)

dots = []
for i in a:
    for j in b:
        dots.append(torch.tensor([i, j],  dtype=torch.float32, device=device).unsqueeze(0))
# dots

new = []
dot = torch.stack(dots)
# dot.shape
for i in dot:
    i = i.to(device)
    new.append(bottleneck.decode(i).squeeze(0))
    


In [21]:
# new = torch.stack(new)
# print(new.shape)
torch.save(new, '/home/dhlee/code/ig_inversion/results/baseline/cifar10_baseline_2.pt')

pca

In [ ]:
from tqdm import tqdm

# bottleneck = torch.load("/home/dhlee/code/ig_inversion/ig_pkg/bottleneck_ae/results/cifar10_8/model_best.pt",  map_location='cpu').eval().to(device)
latent = []
for i in tqdm(valid_dataset):
    input, label = i
    input = input.to(device)    
    x_hat, loss_dict, info = bottleneck(input.unsqueeze(0))
    latent.append(info['bottleneck'])
    # print(info['bottleneck'].shape)
    # print(x_hat.shape)
    # print(latent['bottleneck'].shape)
    # if i ==2: break
latent = torch.stack(latent)
# print(latent.shape)

In [8]:
# latent = torch.stack(latent)
# len(latent)
latent.shape

torch.Size([10000, 1, 8])

In [9]:
a = torch.load('/home/dhlee/code/ig_inversion/results/baseline/cifar10_baseline_2.pt')
a[0].shape

torch.Size([3, 32, 32])

In [14]:
from sklearn.decomposition import PCA

num = latent.clone().detach().cpu().numpy()
num = num.reshape(-1, 8)

pca = PCA(n_components=2)
pca.fit(num)


<bound method BaseEstimator.get_params of PCA(n_components=2)>

In [16]:
pca.get_params(['deep'])

{'copy': True,
 'iterated_power': 'auto',
 'n_components': 2,
 'n_oversamples': 10,
 'power_iteration_normalizer': 'auto',
 'random_state': None,
 'svd_solver': 'auto',
 'tol': 0.0,
 'whiten': False}

In [21]:
# latent1, latent2 = pca.components_
pca.components_

array([[ 0.01552928,  0.571816  , -0.03334171, -0.40915   ,  0.1417619 ,
         0.08252704,  0.2810632 , -0.63116264],
       [-0.06604326, -0.18297149,  0.04879477,  0.7596098 , -0.00115259,
        -0.06883926,  0.28585574, -0.544351  ]], dtype=float32)

In [29]:
a = [-1.0, -0.5, 0.0, 0.5, 1.0]
b = [-1.0, -0.5, 0.0, 0.5, 1.0]

new_latent = []
for i in a:
    for j in b:
        temp = i * latent1 + j * latent2
        new_latent.append(torch.tensor(temp))
# for 
# latent1

In [31]:
new_st = torch.stack(new_latent)
print(new_st.shape)
# new_latent[0]

torch.Size([25, 8])


In [35]:
# c = torch.zeros((1, 2))
ch = bottleneck.decode(new_st.to(device))
ch.shape

torch.Size([25, 3, 32, 32])

In [36]:
torch.save(ch, '/home/dhlee/code/ig_inversion/results/baseline/cifar10_baseline_8.pt')